## 第三章 K近邻法（KNN）
* https://baike.baidu.com/item/k%E8%BF%91%E9%82%BB%E7%AE%97%E6%B3%95/9512781?fr=aladdin  （算法讲这个）
* https://www.cnblogs.com/21207-iHome/p/6084670.html

* KNN可用于分类和回归，用于分类时是多分类方法。
* KNN法1968年由Cover和Hart提出，此方法没有学习训练过程。
* 重要说明1：由于方法用到了距离，而且距离值直接影响到聚类效果，因此在聚类前必须对每一个特征数据进行标准化或归一化处理。
* 重要说明2：由于此方法根据预测点近邻的各类点的个数多少来确定该预测点的类别，因此原始类别数据不均衡，将严重影响最终分类效果。

### 1.KNN方法
* KNN分类主要思想：选择距离未知样本最近的K个训练样本，该K个样本大多数属于某一类型（可考虑距离加权），则未知样本判定为该类型。
* KNN回归主要思想：选择距离未知样本最近的K个训练样本，该K个样本对应的值的平均值就是该未知样本的预测值（求平均值时可考虑距离加权）。
* KNN三要素：k值选择、距离选择、分类(回归)决策规则(分类一般用投票表决，回归一般用均值)
* 关键点：k值的选择问题，k值太小时，模型过于复杂，会出现过拟合；k值过大，模型过于简单，训练结果和预测结果都会不准确，即出现欠拟合。实际应用中，k值一般先取一个较小的值，然后采用交叉验证的方法来选取最优的k值。
* k近邻法实现问题：k近邻法最简单的实现就是线性扫描，这时要计算输入实例与每一个训练实例的距离，当训练集很大时，计算量太大。为了提高k近邻方法搜索的效率，考虑使用特殊结构存储训练数据，以减少计算距离的次数，具体方法很多，李航书中介绍了kd树方法（P43）。

### 2.实例1：使用KNN完成乳腺癌检测分类

#### 乳腺癌检测分类数据集说明
* 乳腺癌检测数据集：数据集共有569个样本，每个样本有30个特征，其中357个阳性，212个阴性。
特征名称意义：  
![caption](./data_picture/chapter3/cancer_data.png)

In [1]:
#1.数据读取
import numpy as np
import pandas as pd
data=pd.read_csv('./data_picture/chapter3/breast-cancer.csv')
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [2]:
#生成训练集和测试集
X=data.drop('class',axis=1)
y=data['class']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=33)

In [3]:
#2.训练KNN模型
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

#数据标准化（如果不考虑舍入误差的影响，对特征和label的数据标准化只影响速度，不影响预测效果）
ss=StandardScaler()
scale=ss.fit(X_train)
X_train=scale.transform(X_train)
X_test=scale.transform(X_test)

#建立模型并预测
model=KNeighborsClassifier()  
model.fit(X_train,y_train)#此时y_train的值是字符型，k近邻方法能自动处理非数值型label列。
                        #sklearn中的分类器应该都能有这个功能，但特征列中的非数值型数据需要自编程序处理为数值型数据

KNeighborsClassifier()

In [4]:
#3.模型评估
print("训练集的模型评估指标：")
y_train_predict=model.predict(X_train)
model_report1=classification_report(y_train,y_train_predict)
print(model_report1)
print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

print("测试集的模型评估指标：")
y_predict=model.predict(X_test)
model_report=classification_report(y_test,y_predict)
print(model_report)
print('--------------------------------------------------------------------------')

训练集的模型评估指标：
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       158
           1       0.97      0.99      0.98       268

    accuracy                           0.97       426
   macro avg       0.97      0.97      0.97       426
weighted avg       0.97      0.97      0.97       426

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
测试集的模型评估指标：
              precision    recall  f1-score   support

           0       0.98      0.93      0.95        54
           1       0.96      0.99      0.97        89

    accuracy                           0.97       143
   macro avg       0.97      0.96      0.96       143
weighted avg       0.97      0.97      0.96       143

--------------------------------------------------------------------------


In [13]:
#4.模型保存
import joblib
joblib.dump(model,'model_knn.pkl')  #保存模型

['model_knn.pkl']

In [15]:
#5.KNN模型预测
from sklearn.preprocessing import StandardScaler

#模型导入
##利用调入的模型对数据进行预测时，要保证待预测数据和模型训练时的数据格式一致，例如如果训练时数据做了标准化，
import joblib                                                              #则待预测数据必须做相应的标准化。
model=joblib.load('model_knn.pkl')     #调入模型

#数据标准化（如果不考虑舍入误差的影响，对特征和label的数据标准化只影响速度，不影响预测效果）
ss=StandardScaler()
scale=ss.fit(X_train)
X_train=scale.transform(X_train)
X_test=scale.transform(X_test)

#模型预测
y_pred=model.predict(X_test)       

#### 注：关于分类模型评价指标说明：
![caption](./data_picture/chapter3/pgt1.jpg)
![caption](./data_picture/chapter3/pgt2.jpg)
![caption](./data_picture/chapter3/pgt3.jpg)

### 3.实例2：利用KNN对美国波士顿地区房价数据进行回归预测

　　该数据集是一个回归问题。每个类的观察值数量是均等的，共有 506 个观察，13 个输入变量和1个输出变量。  
　　每条数据包含房屋以及房屋周围的详细信息。其中包含城镇犯罪率，一氧化氮浓度，住宅平均房间数，到中心区域的加权距离以及自住房平均房价等等。  

　　CRIM：城镇人均犯罪率。  
　　ZN：住宅用地超过 25000 sq.ft. 的比例。  
　　INDUS：城镇非零售商用土地的比例。  
　　CHAS：查理斯河空变量（如果边界是河流，则为1；否则为0）。  
　　NOX：一氧化氮浓度。  
　　RM：住宅平均房间数。  
　　AGE：1940 年之前建成的自用房屋比例。  
　　DIS：到波士顿五个中心区域的加权距离。  
　　RAD：辐射性公路的接近指数。  
　　TAX：每 10000 美元的全值财产税率。  
　　PTRATIO：城镇师生比例。  
　　B：1000（Bk-0.63）^ 2，其中 Bk 指代城镇中黑人的比例。  
　　LSTAT：人口中地位低下者的比例。  
　　MEDV：自住房的平均房价，以千美元计。  

In [1]:
#1.数据读入
import numpy as np
import pandas as pd
data=pd.read_csv('./data_picture/chapter3/boston_house_prices.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [2]:
#2.生成训练集和测试集
from sklearn.model_selection import train_test_split
X=data.drop('MEDV',axis=1)   #生成特征集
y=data['MEDV']               #生成labels集
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=33,test_size=0.25)  #生成训练集和测试集

In [3]:
#3.数据标准化处理
from sklearn.preprocessing import StandardScaler
ss_X=StandardScaler()
scaler_X=ss_X.fit(X_train)
X_train=scaler_X.transform(X_train)
X_test=scaler_X.transform(X_test)

#4.建立KNN回归模型
from sklearn.neighbors import KNeighborsRegressor
model=KNeighborsRegressor(n_neighbors=5)
model.fit(X_train,y_train)   #y_train可以是行向量也可以是列向量，knn会自动将y_train转换为列向量。


KNeighborsRegressor()

In [4]:
#5.模型评估（模型自带评估模块就是做好的评估指标）
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error 
print('训练集回归评估指标：')

y_train_predict=model.predict(X_train)
R2_train=r2_score(y_train,y_train_predict)  #拟合优度值
print('The value of R2:','R2=',R2_train)
mse=mean_squared_error(y_train,y_train_predict)   #均方误差
print('The value of mean_squared_error:','MSE=',mse)
mae=mean_absolute_error(y_train,y_train_predict)  #平均绝对值误差
print('The value of mean_absolute_error:','MAE=',mae)

print('---------------------------------------------------------------------------')
print('测试集回归评估指标：')
y_test_predict=model.predict(X_test)
R2_test=r2_score(y_test,y_test_predict)  #拟合优度值
print('The value of R2:','R2=',R2_test)
mse=mean_squared_error(y_test,y_test_predict)   #均方误差
print('The value of mean_squared_error:','MSE=',mse)
mae=mean_absolute_error(y_test,y_test_predict)  #平均绝对值误差
print('The value of mean_absolute_error:','MAE=',mae)

训练集回归评估指标：
The value of R2: R2= 0.8581070787652896
The value of mean_squared_error: MSE= 12.21917678100264
The value of mean_absolute_error: MAE= 2.2037994722955143
---------------------------------------------------------------------------
测试集回归评估指标：
The value of R2: R2= 0.6907212176346005
The value of mean_squared_error: MSE= 23.981877165354334
The value of mean_absolute_error: MAE= 2.9650393700787396


#### 注：关于回归模型评价指标说明：
![caption](./data_picture/chapter3/pgt8.png)

In [21]:
#6.对新的未知数据做预测
#此处做预测一定要把数据逆变换回去，因为模型预测的结果是标准化后的数据。

new_data=np.array([[0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467, 5.,311,15.2,392.52,20.45],
                   [0.3489,11.5,7.7,0,0.526,6.477,94.3,16.3467, 5.,313,15.2,392.55,20.45]])
X_new=scaler_X.transform(new_data) #标准化
y_new=model.predict(X_new)        #预测
print(y_new)

[20.1 18.1]
